In [ ]:
! pip install dabl
import dabl

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import time
import datetime
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import GroupKFold,train_test_split,cross_validate
from sklearn import tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.model_selection import KFold


In [ ]:
# Load Datasets
df_crash_general = pd.read_csv("/content/drive/My Drive/Datathon/crash_info_general.csv")
df_crash_flag = pd.read_csv("/content/drive/My Drive/Datathon/crash_info_flag_variables.csv")
df_crash_vehicles = pd.read_csv("/content/drive/My Drive/Datathon/crash_info_vehicles.csv")

In [ ]:
# Merge datasets based on CRN and drop columns
general_mod = df_crash_general.iloc[:,[1,2,12,14,15,18,19,20,21,35,36,38,39,42,44,48,61,63,69,70,71,72,80,95,96,98,99,100]]
flag_mod = df_crash_flag.drop(columns=['FATAL','FATAL_OR_SUSP_SERIOUS_INJ','FIRE_IN_VEHICLE','POSSIBLE_INJURY','PROPERTY_DAMAGE_ONLY','INJURY_OR_FATAL'])
df_merged = pd.merge(flag_mod,general_mod,on='CRN')
vehicles_mod = df_crash_vehicles.drop(columns=['AVOID_MAN_CD','COMM_VEH','EMERG_VEH_USE_CD','PRIN_IMP_PT','UNDER_RIDE_IND','VEH_REG_STATE'])
df_merged = pd.merge(df_merged, vehicles_mod, on='CRN')

# Select only vehicles that are 1.0: Striking or 3.0: Striking and Struck
df_merged = df_merged[df_merged["VEH_ROLE"].isin([1.0,3.0])]

# Select only non-secondary crashes
df_merged = df_merged[df_merged["SECONDARY_CRASH"] != "Y"]

# Drop more columns
df_merged = df_merged.drop(columns = ['ARRIVAL_TM',"DISPATCH_TM" ,"TIME_OF_DAY", "MAKE_CD","MODEL_YR","TRAVEL_DIRECTION",'PARTIAL_VIN','VINA_BODY_TYPE_CD','DAMAGE_IND','INJURY','SUSPECTED_MINOR_INJURY','SUSPECTED_SERIOUS_INJURY'])
df_merged.dropna(subset=['DEC_LAT', 'DEC_LONG', 'fips','HOUR_OF_DAY','TRAVEL_SPD' ], inplace= True)

In [ ]:
# Replacing null values with either 0 or the respective unknown values
cols_to_replace = ['SECONDARY_CRASH','INTERSECTION_RELATED']
df_merged[cols_to_replace] = df_merged[cols_to_replace].fillna(0)
cols_to_replace2 = ['SCH_ZONE_IND','TFC_DETOUR_IND','TRL_VEH_CNT']
df_merged[cols_to_replace2] = df_merged[cols_to_replace2].fillna(0)
cols_to_replace0 = ['HORSE_BUGGY','LIMIT_70MPH','INS_IND','TRL_VEH_CNT']
df_merged[cols_to_replace0] = df_merged[cols_to_replace0].fillna(0)
cols_to_replace9 = ['ILLUMINATION','TCD_FUNC_CD','GRADE','RDWY_ALIGNMENT']
df_merged[cols_to_replace9] = df_merged[cols_to_replace9].fillna(9)
cols_to_replace99 = ['BODY_TYPE','IMPACT_POINT','OWNER_DRIVER','SPECIAL_USAGE','VEH_COLOR_CD','VEH_MOVEMENT','VEH_POSITION','VEH_TYPE']
df_merged[cols_to_replace99] = df_merged[cols_to_replace99].fillna(99)
cols_to_replace1 = ['DVR_PRES_IND']
df_merged[cols_to_replace1] = df_merged[cols_to_replace1].fillna(1)

# Standardising string data type to numeric
df_merged.replace({'Y':1,'N':0, 'U':-1},inplace=True)

df_merged.head()

In [ ]:
# Keeping only estimated travel speed of less than 200mph
data_filtered_speed = df_merged[df_merged['TRAVEL_SPD']<200]
data_filtered_speed.info() 

In [ ]:
# Binary encoding of weather data 
df_merged['Blowing Sand'] = np.where(df_merged['WEATHER1' or 'WEATHER2'] == 1, 1, 0)
df_merged['Blowing Snow'] = np.where(df_merged['WEATHER1' or 'WEATHER2'] == 2, 1, 0)
df_merged['Clear'] = np.where(df_merged['WEATHER1' or 'WEATHER2'] == 3, 1, 0)
df_merged['Cloudy'] = np.where(df_merged['WEATHER1' or 'WEATHER2'] == 4, 1, 0)
df_merged['Fog/Smog/Smoke'] = np.where(df_merged['WEATHER1' or 'WEATHER2'] == 5, 1, 0)
df_merged['Freezing Rain'] = np.where(df_merged['WEATHER1' or 'WEATHER2'] == 6, 1, 0)
df_merged['Rain'] = np.where(df_merged['WEATHER1' or 'WEATHER2'] == 7, 1, 0)
df_merged['Severe Crosswinds'] = np.where(df_merged['WEATHER1' or 'WEATHER2'] == 8, 1, 0)
df_merged['Sleet/Hail'] = np.where(df_merged['WEATHER1' or 'WEATHER2'] == 9, 1, 0)
df_merged['Snow'] = np.where(df_merged['WEATHER1' or 'WEATHER2'] == 10, 1, 0)
df_merged.head()

In [ ]:
duplicate_ids = df_merged[df_merged.duplicated(subset='CRN', keep=0)]['CRN'].unique()
duplicate_ids.size #Number of duplicated CRNs

In [ ]:
df_all_int = df_merged.drop(columns=["LATITUDE","LONGITUDE"])
df_merged = df_all_int
pd.set_option('display.max_rows',None)
print(df_all_int.info())
print(df_all_int.dtypes)

In [ ]:
# Feature Engineering 
# Hour Of Day, Illumination, Vehicle Body Type, Driver Presence Indicator, Backup

# Binary encoding for Hour Of Day
dawn = df_merged['HOUR_OF_DAY'] >= 6
dusk = df_merged['HOUR_OF_DAY'] <= 18
df_merged['DAY'] = np.where(dawn & dusk , 1, 0)

# Drop other/unknown illumination
df_merged = df_merged[df_merged['ILLUMINATION'] != (8 or 9)]
# Daylight, Dimmed (Dusk/dawn), Dark (with streetlight/other lighting), Dark
brightness_cond = [df_merged['ILLUMINATION'] == 1,
                   df_merged['ILLUMINATION'] == (4 or 5),
                   df_merged['ILLUMINATION'] == (3 or 6), 
                   df_merged['ILLUMINATION'] == 2]
brightness_out = [4,3,2,1]   
df_merged['BRIGHTNESS'] = np.select(brightness_cond,brightness_out)

# Re-categorise vehicle body type into: Car, Motorcycle, Bus, Van, Trucks/Heavy duty represented by 1-5
df_merged = df_merged.drop(columns =["MOTORCYCLE","SCHOOL_BUS",'HVY_TRUCK_RELTED'],axis = 1)
veh_cond = [df_merged['BODY_TYPE'] >= 1, df_merged['BODY_TYPE'] <=19, 
            df_merged['BODY_TYPE'] >= 20, df_merged['BODY_TYPE'] <=29,
            df_merged['BODY_TYPE'] >= 30, df_merged['BODY_TYPE'] <=39,
            df_merged['BODY_TYPE'] >= 40, df_merged['BODY_TYPE'] <=49,
            df_merged['BODY_TYPE'] >= 50, df_merged['BODY_TYPE'] <=79]
veh_out = [1,1,2,2,3,3,4,4,5,5] 
df_merged['4-WHEEL'] = np.where(veh_cond[0] & veh_cond[1], 1, 0)
df_merged['MOTORCYCLE'] = np.where(veh_cond[2] & veh_cond[3], 1, 0)  
df_merged['BUS'] = np.where(veh_cond[4] & veh_cond[5], 1, 0)  
df_merged['VAN'] = np.where(veh_cond[6] & veh_cond[7], 1, 0) 
df_merged['HEAVY_VEHICLES'] = np.where(veh_cond[8] & veh_cond[9], 1, 0)  

# Creating new features 'DRUNK/DRUGGED' and 'SICK/FATIGUED/ASLEEP' 
sober_cond = [df_merged['DVR_PRES_IND'] == 2 ,df_merged['DVR_PRES_IND'] == 3, df_merged['DVR_PRES_IND'] == 7,
              df_merged['DVR_PRES_IND'] == 4, df_merged['DVR_PRES_IND'] == 5, df_merged['DVR_PRES_IND'] == 6]
df_merged['DRUNK/DRUGGED'] = np.where(sober_cond[0] | sober_cond[1] | sober_cond[2] , 1, 0)  
df_merged['SICK/FATIGUED/ASLEEP'] = np.where(sober_cond[3] | sober_cond[4] | sober_cond[5], 1, 0)  
#Removing duplicate columns: 'IMPAIRED_DRIVER', 'FATIGUE_ASLEEP'
df_merged = df_merged.drop(columns = ["IMPAIRED_DRIVER","FATIGUE_ASLEEP"],axis = 1)

# Merging all 3 Backup columns into single BACKUP column
traffic_cond = (df_merged['BACKUP_PRIOR'] == 1) | (df_merged['BACKUP_NONRECURRING'] == 1) | (df_merged['BACKUP_CONGESTION'] == 1)
df_merged['BACKUP'] = np.where(traffic_cond, 1, 0)

# Remove all additional columns
df_merged = df_merged.drop(columns = ['BACKUP_PRIOR', 'BACKUP_NONRECURRING', 'BACKUP_CONGESTION','WEATHER1','WEATHER2','SECONDARY_CRASH'],axis =1 )  

In [ ]:
# Remove all vehicle columns that were not merged
end_id = df_merged.columns.get_loc('BODY_TYPE')
start_id = df_merged.columns.get_loc('Blowing Sand')
TRAVEL_SPD = df_merged.TRAVEL_SPD
INS_IND = df_merged.INS_IND
df_merged_rowsdeleted = df_merged.drop(df_merged.columns[end_id:start_id],axis = 1)
df_merged_rowsdeleted["TRAVEL_SPD"] = TRAVEL_SPD
df_merged_rowsdeleted["INS_IND"] = INS_IND

In [ ]:
# Remove duplicate CRNs, keeping vehicle info and max speed
df_merged_rowsdeleted = df_merged_rowsdeleted[df_merged_rowsdeleted['TRAVEL_SPD'] < 200]

def remove_duplicates(group) :
    merged_row = group.loc[group['TRAVEL_SPD'].idxmax()]
    #merged_row = group.loc[group['LANE_COUNT'].idxmax()]
    
    # Add a 1 for columns 'car', 'motorcycle', 'truck', 'van', 'bus'
    # if they are True for the duplicate rows
    vehicle_columns = ['4-WHEEL', 'MOTORCYCLE', 'BUS', 'VAN', 'HEAVY_VEHICLES', 'DRUNK/DRUGGED', 'SICK/FATIGUED/ASLEEP','INS_IND']
    for col in vehicle_columns:
        merged_row[col] = np.where(group[col].any(), 1, merged_row[col])
    return merged_row

df_noDuplicates = df_merged_rowsdeleted.groupby('CRN',sort=True,as_index=False).apply(remove_duplicates)

In [ ]:
# Drop unknown severity and unknown in injured
df_noDuplicates = df_noDuplicates[(df_noDuplicates['MAX_SEVERITY_LEVEL'] != 8) & (df_noDuplicates['MAX_SEVERITY_LEVEL'] != 9)]

# Replace unknwon brightness with 3(BRIGHT) if clear and day, replace with 1 (Dark) if clear and night
df_noDuplicates.loc[(df_noDuplicates['Clear'] == 1) & (df_noDuplicates['DAY'] == 1) & (df_noDuplicates['BRIGHTNESS'] == -1), 'BRIGHTNESS'] = 3
df_noDuplicates.loc[(df_noDuplicates['Clear'] == 1) & (df_noDuplicates['DAY'] == 0) & (df_noDuplicates['BRIGHTNESS'] == -1), 'BRIGHTNESS'] = 1

severity_reorder = {1:3, 2:2, 3:1, 4:1}
df_noDuplicates['MAX_SEVERITY_LEVEL'].replace(severity_reorder, inplace=True)
df_noDuplicates.head()

In [ ]:
# Sort by date and output data to CSV
df_noDuplicates.head()
df_noDuplicates.to_csv('cleaned.csv')